In [1]:
# read in known applicant files, clean and try to dedupe
from collections import Counter
import difflib
from fuzzywuzzy import fuzz
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os


from data_cleaning_functions import correct_mispellings, long_form_date, clean_names, clean_med_school
from dev import (
    SUFFIXES, FEMALE_FIRST_NAMES, FEMALE_MIDDLE_NAMES, NAME_COLS, RAW_ATT_DATA_DIR, ATT_DATA_DIR, PICKLE_DIR, 
        CORRECTIONS_DIR, SUM_STAT_DIR)


%load_ext autoreload
%autoreload 2

%reload_ext autoreload

In [2]:
r1_file = '1964-1973 associates.XLS'
r2_file = 'Associates alpha by institute.XLS'
r3_file = 'Associates data.XLS'
r4_file = 'NIMH Associates Complete.XLS'
r5_file = 'NINDB Associates alpha by year.xls'
filenames = [r1_file, r2_file, r3_file, r4_file, r5_file]

In [3]:
file_df = map(lambda x: pd.read_excel(os.path.join(RAW_ATT_DATA_DIR, x)), filenames)

file_4_columns = [
    'dno', 'source', 'unknown', 'lastname', 'first_middle', 'institute', 'lab_brch', 
    'program', 'supervisor', 'eod_year', 'med_school', 'year_grad', 'intern_hos', 'intern_dte',
       'res_hosp', 'residency', 'res_dtes'
]

file_df[2].rename(columns={'lname':'lastname', 'fname': 'first_middle'}, inplace=True)
file_df[1].rename(columns={'lname':'lastname', 'fname': 'first_middle'}, inplace=True)

file_df[4].columns = file_4_columns
# for each files in the list, add a column to track source
for name, f in zip(filenames, file_df):
    f.loc[:, 'data_source'] = name
concat_df = pd.concat(file_df).reset_index(drop=True)

print sum(map(lambda x: x.shape[0], file_df)) == concat_df.shape[0]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
True


In [4]:
def has_suffix(lst_strs):
    # check for any existing suffix
    existing_suffixes = filter(None, filter(lambda x: x in lst_strs, SUFFIXES))
    # if suffix is I or V, only return suffix if len of lst_string > 2 (otherwise middle name)
    if ('I' in existing_suffixes or 'V' in existing_suffixes) and len(lst_strs) > 2:
        return existing_suffixes
    return funcy.remove(lambda x: x in ['I', 'V', 'DR'], existing_suffixes)

In [5]:
def strip_first_middle(raw_str):
    # seperate first_middle column string (Thomas, Bruce) into THOMAS BRUCE
    # suffix also appears to be in column THOMAS BRUCE JR
    names = filter(None, raw_str.split(' '))
    cleaned_vals = map(clean_names, names)
    suff = has_suffix(cleaned_vals)
    # suffix is actually a list
    if suff:
        root = funcy.remove(lambda x: x==suff[-1], cleaned_vals)
        return pd.Series(
            {'clean_first_name': root[0],
                 'clean_middle_name': ' '.join(root[1:]),
                    'clean_suffix': suff[-1]})
    return pd.Series(
        {'clean_first_name': cleaned_vals[0],
             'clean_middle_name': np.nan if len(cleaned_vals)==1 else ' '.join(cleaned_vals[1:]),
                'clean_suffix': np.nan})

In [6]:
cleaned_suffix_df = pd.DataFrame(concat_df.loc[~pd.isnull(concat_df.first_middle), 'first_middle'].apply(strip_first_middle))

In [7]:
def get_suffix(raw_str):
    # wrapper around get suffix
    if pd.isnull(raw_str) or len(raw_str.split(' ')) < 2:
        return np.nan
    str_list = raw_str.split(' ')
    suff = has_suffix(str_list)
    if suff:
        return suff[-1]
    return np.nan

In [8]:
# now we have all the associates, sep first middle into first and middle name, then sort and check 
# to see if we have any duplicates
df2 = pd.concat([concat_df, cleaned_suffix_df], axis=1)
df2.head()
df2.loc[:, 'clean_last_name'] = df2.lastname.apply(clean_names)

In [9]:
# consolidate firstname columns
missing_first_middle = pd.isnull(df2.first_middle)
df2.loc[missing_first_middle, ['clean_first_name', 'clean_middle_name']] = df2.loc[missing_first_middle, ['firstname', 'middlename']].applymap(clean_names)
df2.loc[:, 'len_middle'] = df2.clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) else len(x.split(' ')))

# clean suffix out from any other columns
may_have_suffix_mask = df2.len_middle > 1
df2.loc[may_have_suffix_mask, 'clean_suffix'] = df2[may_have_suffix_mask]['clean_middle_name'].apply(get_suffix)

df3 = df2.drop(['first_middle', 'firstname', 'middlename', 'lastname', 'len_middle'], axis=1)
# df3 = df2
# # dropnow where both first and last name are missing
df4 = df3.dropna(subset=['clean_first_name', 'clean_last_name'], how='all')
print df3.shape
print df4.shape

(10729, 24)
(10723, 24)


In [10]:
# remove all females
female_mask = np.logical_or(
    df4.clean_first_name.isin(FEMALE_FIRST_NAMES), df4.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))

df5 = df4.loc[~female_mask, :].sort_values(NAME_COLS)
df5.loc[:, 'clean_middle_initial'] = df5.clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) or x=='' else x[0])
df5.loc[:, 'clean_first_initial'] = df5.clean_first_name.apply(lambda x: np.nan if pd.isnull(x) or x=='' else x[0])
df5.loc[:, 'clean_medical_school'] = df5.med_school.apply(funcy.rcompose(clean_names, clean_med_school))

KING SAUD UNIVERSITY
KING SAUD UNIVERSITY
CHARLES UNIVERSITY
CHARLES UNIVERSITY
MCGILL UNIVERSITY
MCGILL UNIVERSITY
CAIRO UNIVERSITY MEDICAL SCHOOL
CAIRO UNIVERSITY MEDICAL SCHOOL
ST GEORGE S UNIVERSITY GRENADA
ST GEORGE S UNIVERSITY GRENADA
TAIPEI MEDICAL COLLEGE TAIPEI TAIWAN
TAIPEI MEDICAL COLLEGE TAIPEI TAIWAN
NEWCASTLE UPON TYNE MEDICAL SCHOOL ENGLAND
NEWCASTLE UPON TYNE MEDICAL SCHOOL ENGLAND
UNIVERSITY OF ZAMBIA
UNIVERSITY OF ZAMBIA
MADRAS MEDICAL COLLEGE INDIA
MADRAS MEDICAL COLLEGE INDIA
BUCKNELL UNIVERSITY
BUCKNELL UNIVERSITY
SOUTHWESTERN MEDICAL SCHOOL
SOUTHWESTERN MEDICAL SCHOOL
PONCE SCHOOL OF MEDICINE
PONCE SCHOOL OF MEDICINE
G S V M MEDICAL COLLEGE KANPUR
G S V M MEDICAL COLLEGE KANPUR
SACKLER TEL AVIV UNIVERSITY ISRAEL
SACKLER TEL AVIV UNIVERSITY ISRAEL
DOW MEDICAL COLLEGE KARACHI UNIVERSITY
DOW MEDICAL COLLEGE KARACHI UNIVERSITY
UNIVERSITY OF OSTEOPATHIC MEDICINE HEALTH SCIENCE
UNIVERSITY OF OSTEOPATHIC MEDICINE HEALTH SCIENCE
MCGILL UNIVERSITY
MCGILL UNIVERSITY
UNIVER

In [26]:
def count_missing(row):
    # count the number of entries that are null
    return row[pd.isnull(row)].size

df5.loc[:, 'count_missing'] = df5[
    NAME_COLS+['clean_medical_school', 'eod_year', 'res_dtes', 'res_hosp', 'year_grad']].apply(count_missing, axis=1)

In [27]:
# drop duplicates by dno
df6 = df5.drop_duplicates('dno').sort_values(
    ['clean_last_name', 'clean_first_initial', 'clean_middle_name', 'count_missing'], axis=0)
print df5.shape
print df6.shape

(10234, 28)
(3909, 28)


In [28]:
man = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_dno_matches.xlsx'), index=False).rename(columns={'medical_school': 'dno_medical_school'})
man['medical_school'] = man.dno_medical_school.apply(clean_med_school)

WASHINGTON
WASHINGTON


In [29]:
df_merge = df6.drop(NAME_COLS, axis=1)

In [30]:
df_  = pd.merge(left=df_merge, right=man, on=['dno'], how='inner')
print df_.shape
print man.shape
# all missing are femal


(100, 32)
(104, 8)


In [31]:
df6_ = df_.combine_first(df6)

In [32]:
# reassign name of df6 and continue with merge
df6 = df6_

# need to fill in missing first, middle and last names

In [33]:
df6_.loc[df6_.clean_last_name=='FRAM']

,application_year_max,application_year_min,citizenship,clean_first_initial,clean_first_name,clean_last_name,clean_medical_school,clean_middle_initial,clean_middle_name,clean_suffix,...,medical_school,program,res_dtes,res_hosp,residency,source,ssn,supervisor,unknown,year_grad
24,1964.0,1964.0,NaN,NaN,DAVID,FRAM,YALE UNIVERSITY SCHOOL OF MEDICINE,NaN,H,NaN,...,YALE UNIVERSITY SCHOOL OF MEDICINE,NaN,1965-1968,Massachusetts Mental Health Center,Psychiatry,NIH EDUCATION,NaN,NaN,NaN,1964.0
10547,NaN,NaN,NaN,NaN,NaN,FRAM,YALE UNIVERSITY SCHOOL OF MEDICINE,NaN,NaN,NaN,...,NaN,NaN,1965-1968,Massachusetts Mental Health Center,Psychiatry,NIH EDUCATION,NaN,NaN,NaN,1964.0


In [34]:
def get_year_diff(x):
    # get reference value
    ref_year = x.iloc[0]
    return pd.Series(x - ref_year)

def is_duplicate(grouped_df):
    # return true if grouped df should be counted as one duplicate, returns filter mask
    # check if med schools match/are missing
    # check if eod years match/are missing
    # first group_id
    if grouped_df.shape[0] == 1:
        return pd.Series([np.nan])
    med_schools = grouped_df['clean_medical_school'].dropna().unique()
    # if only <= 1 unique med school, mark as duplicate
    ref_dno = grouped_df['dno'].values[0]
    if len(med_schools) < 2:
        return pd.Series([ref_dno]*grouped_df.shape[0])
    ms1 = med_schools[0]
    # otherwise, check the string similarity of the med school
    med_school_sims = [
        np.nan if pd.isnull(ms) else fuzz.partial_token_sort_ratio(
                ms1, ms) for ms in grouped_df.clean_medical_school.values]
    # for med schools with high string sim, count as dups, otherwise not 
    is_dups = [ref_dno if pd.isnull(sim) or sim > .8 else False for sim in med_school_sims]
    return pd.Series(is_dups)
    

In [35]:
df6.loc[df6.dno==4130, :]
df6.loc[df6.clean_last_name=='ZIVIN', :]
df6.loc[df6.clean_last_name=='BROWN', :]

,application_year_max,application_year_min,citizenship,clean_first_initial,clean_first_name,clean_last_name,clean_medical_school,clean_middle_initial,clean_middle_name,clean_suffix,...,medical_school,program,res_dtes,res_hosp,residency,source,ssn,supervisor,unknown,year_grad
10,1975.0,1975.0,NaN,E,ERIC,BROWN,HARVARD MEDICAL SCHOOL,J,JOEL,NaN,...,HARVARD MEDICAL SCHOOL,NaN,1976-1977,Beth Israel - Boston,Medicine,NIH EDUCATION,NaN,NaN,NaN,1975.0
11,1966.0,1966.0,NaN,J,JAMES,BROWN,INDIANA UNIVERSITY SCHOOL OF MEDICINE,W,EDWARD,NaN,...,YALE UNIVERSITY SCHOOL OF MEDICINE,CA,NaN,NaN,NaN,NIH EDUCATION,NaN,Morrow,NaN,1970.0
12,1967.0,1967.0,NaN,W,WALTER,BROWN,DUKE MEDICAL SCHOOL,A,ARMIN,NaN,...,DUKE MEDICAL SCHOOL,NaN,1968-1969,Yale Psychiatric Institute,Psychiatry,NIH EDUCATION,NaN,Chapman,NaN,1967.0
2180,NaN,NaN,NaN,C,CLARENCE,BROWN,EMORY UNIVERSITY SCHOOL OF MEDICINE,H,H,III,...,NaN,CA,1967-1968,Johns Hopkins Hospital,Medicine,NIH EDUCATION,NaN,NaN,NaN,1966.0
2181,NaN,NaN,NaN,J,JAMES,BROWN,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE,A,ALAN,NaN,...,NaN,NaN,1968-1969,"Presbyterian Hospital, NYC",Medicine,NIH EDUCATION,NaN,Rall,NaN,1967.0
2182,NaN,NaN,NaN,R,RICHARD,BROWN,STANFORD UNIVERSITY,J,JOEL,NaN,...,NaN,SA,NaN,NaN,NaN,NIH EDUCATION,NaN,Dingman,NaN,1968.0
2183,NaN,NaN,NaN,R,ROBERT,BROWN,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,S,S,NaN,...,NaN,CA,1964-1965,Bellevue Hospital,Medicine,NIH EDUCATION,NaN,Carbone,NaN,1963.0
2184,NaN,NaN,NaN,W,WILLIS,BROWN,VANDERBILT UNIVERSITY SCHOOL OF MEDICINE,E,E,JR,...,NaN,CA,1964-1965,Vanderbilt,Surgery,NIH EDUCATION,NaN,Ketcham,NaN,1963.0
3210,NaN,NaN,NaN,R,ROBERT,BROWN,UNIVERSITY OF IOWA COLLEGE OF MEDICINE,S,STICKLER,NaN,...,NaN,NaN,1969-1971,"University Hospital, Iowa City, IA",Ophthalmology,NIH EDUCATION,NaN,NaN,NaN,1968.0
3258,NaN,NaN,NaN,B,BRUCE,BROWN,JOHNS HOPKINS UNIVERSITY SCHOOL OF MEDICINE,G,GREGORY,NaN,...,NaN,NaN,NaN,NaN,Medicine,NIH EDUCATION,NaN,NaN,NaN,1969.0


In [36]:
df6_grouped = df6.groupby(['clean_last_name', 'clean_first_initial'])
df6['eod_year_diff'] = df6_grouped.eod_year.apply(get_year_diff)
# df6['is_dup_index'] = df6_grouped.apply(is_duplicate)
new_col = df6_grouped.apply(is_duplicate)
# reset so no longer grouped as a multi index, then merge in via clean first, last_name

# dups = df6_grouped.filter(lambda x: is_duplicate(x)) 

In [37]:
new_col2 = new_col.reset_index()
new_col2.drop('level_2', axis=1, inplace=True)
print new_col.shape
print new_col2.shape


(3958,)
(3958, 3)


In [38]:
new_col2.columns = ['clean_last_name', 'clean_first_initial', 'duplicate_dno']
new_col3 = new_col2.drop_duplicates(['clean_last_name', 'clean_first_initial', 'duplicate_dno'])

In [39]:
new_col4 = new_col3[new_col3['duplicate_dno']!=False]
new_col4[new_col4['clean_last_name']=='AARON']

,clean_last_name,clean_first_initial,duplicate_dno
0,AARON,R,NaN


In [40]:
df7 = pd.merge(left=df6, right=new_col4, on=['clean_last_name', 'clean_first_initial'], how='inner')

In [44]:
# df7.loc[df7.clean_last_name=='FRAM']
df6.loc[df6.clean_last_name=='FRAM']

,application_year_max,application_year_min,citizenship,clean_first_initial,clean_first_name,clean_last_name,clean_medical_school,clean_middle_initial,clean_middle_name,clean_suffix,...,program,res_dtes,res_hosp,residency,source,ssn,supervisor,unknown,year_grad,eod_year_diff
24,1964.0,1964.0,NaN,NaN,DAVID,FRAM,YALE UNIVERSITY SCHOOL OF MEDICINE,NaN,H,NaN,...,NaN,1965-1968,Massachusetts Mental Health Center,Psychiatry,NIH EDUCATION,NaN,NaN,NaN,1964.0,NaN
10547,NaN,NaN,NaN,NaN,NaN,FRAM,YALE UNIVERSITY SCHOOL OF MEDICINE,NaN,NaN,NaN,...,NaN,1965-1968,Massachusetts Mental Health Center,Psychiatry,NIH EDUCATION,NaN,NaN,NaN,1964.0,NaN


In [45]:
df7.loc[
    df7.clean_last_name=='BROWN', NAME_COLS+['clean_medical_school', 'eod_year', 'res_dtes', 'res_hosp', 'year_grad', 'dno']]

,clean_first_name,clean_middle_name,clean_last_name,clean_medical_school,eod_year,res_dtes,res_hosp,year_grad,dno
24,ERIC,JOEL,BROWN,HARVARD MEDICAL SCHOOL,1977.0,1976-1977,Beth Israel - Boston,1975.0,458.0
25,ERIC,JOEL,BROWN,HARVARD MEDICAL SCHOOL,1977.0,1976-1977,Beth Israel - Boston,1975.0,458.0
26,EDWARD,MEIGS,BROWN,HARVARD MEDICAL SCHOOL,1974.0,1973-1974,Peter Bent Brigham Hospital,1972.0,457.0
27,JAMES,EDWARD,BROWN,INDIANA UNIVERSITY SCHOOL OF MEDICINE,1972.0,NaN,NaN,1970.0,462.0
28,JAMES,ALAN,BROWN,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE,1969.0,1968-1969,"Presbyterian Hospital, NYC",1967.0,460.0
29,JOHN,W,BROWN,INDIANA UNIVERSITY SCHOOL OF MEDICINE,1972.0,NaN,NaN,1970.0,462.0
30,JAMES,EDWARD,BROWN,YALE UNIVERSITY SCHOOL OF MEDICINE,1968.0,1967-1968,Yale - New Haven Hospital,1966.0,461.0
31,WALTER,ARMIN,BROWN,DUKE MEDICAL SCHOOL,1969.0,1968-1969,Yale Psychiatric Institute,1967.0,474.0
32,WILLIS,E,BROWN,VANDERBILT UNIVERSITY SCHOOL OF MEDICINE,1965.0,1964-1965,Vanderbilt,1963.0,475.0
33,W,VIRGIL,BROWN,YALE UNIVERSITY SCHOOL OF MEDICINE,1966.0,1965-1966,Johns Hopkins,1964.0,473.0


In [46]:
NONDUP_DNO = (187,  224,  244,  246,  256,  257,  345,  350,  349,  458,  461,
        474,  623,  643,  645,  665,  667,  673,  719,  807,  873,  996,
       1099, 1153, 1163, 1176, 1229, 1237, 1356, 1393, 1541, 1648, 1794,
       1798, 1801, 1882, 1913, 2032, 2058, 2105, 2173, 2183, 2189, 2207,
       2210, 2228, 2258, 2355, 2371, 2400, 2463, 2474, 2542, 2548, 2554,
       2559, 2596, 2653, 2655, 2672, 2866, 3021, 3138, 3140, 3172, 3207,
       3313, 3317, 3320, 3333, 3334, 3373, 3402, 3429, 3453, 3472, 3482,
       3484, 3485, 3490, 3497, 3498, 3510, 3600, 3644, 3646, 3710, 3721,
       3985, 3990, 3993, 4031, 4058, 4090, 4092, 4095)

df7.loc[:, 'eod_year_diff'] = df7.eod_year_diff.abs()
df7.loc[(df7.eod_year_diff>2) , 'duplicate_dno'] = np.nan
df7.loc[df7.dno.isin(NONDUP_DNO) , 'duplicate_dno'] = 0 
df7.loc[df7.duplicate_dno==df7.dno , 'duplicate_dno'] = np.nan

In [47]:
df7.loc[:, 'dup_flag'] = 0
dup_mask = df7.dno.isin(df7.duplicate_dno.dropna())
df7.loc[dup_mask, 'dup_flag'] = 1

In [48]:
missing_flags = ['missing_{}'.format(f) for f in df7.columns]

def count_missing(x):
    # returns 1 for missing value
    if x is None or x in ['None', '', ' ']:
        return 1
    return 1 if pd.isnull(x) else 0

# calculate missing info stats
df7[missing_flags] = df7.applymap(count_missing)

# calculate missing stats for people in the data set
df7[missing_flags].describe().to_csv(os.path.join(SUM_STAT_DIR, 'NIH_attendees_data_fill_rates.csv'))
df8 = df7.drop(missing_flags, axis=1)

In [49]:
def consolidate_row(series):
    nonnulls = series.dropna().unique()
    if nonnulls.shape[0] == 0:
        return pd.Series([np.nan]*series.shape[0])
    if isinstance(nonnulls[0], str):
        val = sorted(nonnulls, key=len, reverse=True)[0]
        return pd.Series([val]*series.shape[0])
    val = sorted(nonnulls, reverse=True)[0]
    return pd.Series([val]*series.shape[0])


In [50]:
grouped_dups = df8.loc[
    df8.duplicated(['clean_last_name', 'clean_first_initial'], keep=False), :].groupby(
        ['clean_last_name', 'clean_first_initial'], as_index=False)
new_df = []
for g, df in grouped_dups:
    consol_df = df.apply(consolidate_row, axis=0)
    consol_df['duplicate_dno'] = [df.dno.values]*df.shape[0]
    new_df.append(consol_df)
    

In [51]:
new_df2 = pd.concat(new_df, axis=0)
new_df3 = new_df2.drop_duplicates(['clean_first_name', 'clean_last_name', 'eod_year'])

In [52]:
missing = df6.loc[~df6.dno.isin(df8.dno),:]
print missing.shape

(51, 33)


In [53]:
# df8.loc[df8.clean_first_name=='WALTER']

In [54]:
df8.loc[df8.duplicate_dno==0, 'duplicate_dno'] = np.nan

In [55]:
#now, merge in manual corrections to add any more duplicate dnos
# finally consolidate people merged into the same row
df9 = pd.concat([df8.loc[pd.isnull(df8.duplicate_dno), :], new_df3, missing], axis=0)


# df7.loc[df7.duplicated(['clean_last_name', 'clean_medical_school'], keep=False), NAME_COLS+['clean_medical_school', 'eod_year', 'dno', 'duplicate_dno']]

In [57]:
df9.loc[df9.clean_last_name=='TEW', NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]
df8.loc[df8.dno==3699, NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]
df8.loc[df8.dno.isin([3700, 3699]), NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]
df8.loc[df8.dno.isin([3700, 3699]), NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]


# df9.loc[df9.duplicated('dno', keep=False), NAME_COLS+['dno', 'dup_flag', 'duplicate_dno']]

df9.loc[df9.clean_last_name=='FRAM', NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]

,clean_first_name,clean_middle_name,clean_last_name,dup_flag,duplicate_dno,dno
24,DAVID,H,FRAM,NaN,NaN,1153.0
10547,NaN,NaN,FRAM,NaN,NaN,1153.0


In [58]:
to_remove = ['TERRECE', 'FRED', 'LAURENCE',
             'CUONO', 'DEFRENZE', 'JEFFERY', 'FINKLEMAN', 'SHERRAD', 'ANSCHNETZ', 'MARC', 'JENSON', 'KASTI', 
            'ADELBERT', 'RITCHARD', 'MANSFORD', 'DEFRENZO', 'DROBIN', 'HAMES', 'KREUZ', 'JERROLD', 'MANEUSI',
            'UNGARO']
to_replace = ['TERRENCE', 'FREDERICK', 'LAWRENCE',
              'CUOMO', 'DEFRONZO', 'JEFFREY', 'FINKELMAN', 'SHERRARD', 'ANSCHUETZ', 'MARCUS', 'JENSEN', 'KASTL',
              'ALBERT', 'RITCHARD', 'MANIFORD', 'DEFRONZO', 'DROBIS', 'JAMES', 'KRUEZ', 'JERROD', 'MANCUSI',
              'UNGARO']

correct_name_mispellings_fnc = funcy.rpartial(correct_mispellings, to_remove, to_replace)

df9.loc[:, 'clean_last_name'] = df9.clean_last_name.apply(correct_name_mispellings_fnc)
df9.loc[:, 'clean_first_name'] = df9.clean_first_name.apply(correct_name_mispellings_fnc)

In [59]:
# function to go in and correct some of the name mispellings in both data sets
# MUTATING FUNCTION
def change_names(df, selection_type, selection_value, to_change_type, to_change_values):
    for t, v in zip(to_change_type, to_change_values):
        print t, v
        df.loc[df[selection_type]==selection_value, t] = v

In [60]:
change_names(
    df9, 'clean_last_name', 'CHESEBRO', ['clean_first_name', 'clean_middle_name'], ['BRUCE', 'WILCOX'])
change_names(df9, 'clean_last_name', 'GALANTER', ['clean_first_name', 'clean_middle_name'], ['MARC', 'I'])
change_names(
    df9, 'clean_last_name', 'BEAN', ['clean_first_name', 'clean_middle_name', 'clean_medical_school'], ['SIDNEY', 'CHARLES', 'WAKE_FOREST'])
change_names(
    df9, 'clean_last_name', 'EILER', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    df9, 'clean_last_name', 'FALCHUK', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])


clean_first_name BRUCE
clean_middle_name WILCOX
clean_first_name MARC
clean_middle_name I
clean_first_name SIDNEY
clean_middle_name CHARLES
clean_medical_school WAKE_FOREST
clean_first_name DONALD
clean_middle_name MARTIN
clean_first_name DONALD
clean_middle_name MARTIN


In [61]:
df9.loc[df9.dno==462]

,application_year_max,application_year_min,citizenship,clean_first_initial,clean_first_name,clean_last_name,clean_medical_school,clean_middle_initial,clean_middle_name,clean_suffix,...,medical_school,program,res_dtes,res_hosp,residency,source,ssn,supervisor,unknown,year_grad
27,1966.0,1966.0,NaN,J,JAMES,BROWN,INDIANA UNIVERSITY SCHOOL OF MEDICINE,W,EDWARD,NaN,...,YALE UNIVERSITY SCHOOL OF MEDICINE,CA,NaN,NaN,NaN,NIH EDUCATION,NaN,Morrow,NaN,1970.0
29,NaN,NaN,NaN,J,JOHN,BROWN,INDIANA UNIVERSITY SCHOOL OF MEDICINE,W,W,NaN,...,NaN,CA,NaN,NaN,NaN,NIH EDUCATION,NaN,Morrow,NaN,1970.0
0,1966.0,1966.0,NaN,J,JOHN,BROWN,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE,W,W,NaN,...,YALE UNIVERSITY SCHOOL OF MEDICINE,CA,1968-1969,Yale - New Haven Hospital,Medicine,NIH EDUCATION,NaN,Rall,NaN,1970.0


In [62]:
# fill in missing eod year
# manual corrections
df9.loc[(df9.clean_last_name=='BEELS') & pd.isnull(df9.eod_year), 'eod_year'] = 1967
df9.loc[(df9.clean_last_name=='KRAUSE') & pd.isnull(df9.eod_year), 'eod_year'] = 1965
df9.loc[(df9.clean_last_name=='MELLMAN') & pd.isnull(df9.eod_year), 'eod_year'] = 1986
# df9.loc[pd.isnull(df9.eod_year), :]

In [63]:
# import manual eod year fixes
manual_eod_fixes = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_eod_fixes.xlsx'))
manual_eod_fixes.head()
manual_eod_df = manual_eod_fixes[['clean_last_name'] + [c for c in manual_eod_fixes.columns if c.startswith('to_fix')]]

missing_eod = pd.merge(
    left=df9.loc[pd.isnull(df9.eod_year), :], right=manual_eod_df, on='clean_last_name', how='inner')

for x in ['clean_medical_school', 'clean_first_name', 'clean_middle_name', 'eod_year']:
    mask = ~pd.isnull(missing_eod['to_fix_{}'.format(x)])
    missing_eod.loc[mask, x] = missing_eod.loc[mask, 'to_fix_{}'.format(x)]


In [64]:
missing_eod2 = missing_eod.drop(
    (c for c in missing_eod.columns if c not in df9.columns), axis=1).drop_duplicates(['clean_last_name'])

In [65]:
df10 = pd.concat([df9.loc[~pd.isnull(df9.eod_year), :], missing_eod2], axis=0).sort_values(NAME_COLS, axis=0)
# drop duplicates on dno

In [66]:
df10A = df10.drop_duplicates('dno', keep='first')

In [67]:
dups = df10A.loc[df10A.duplicated(NAME_COLS, keep=False),:]
# df10A.loc[df10A.duplicated(NAME_COLS, keep=False), NAME_COLS+['duplicate_dno', 'dno']]

In [68]:
non_dups = dups[~pd.isnull(dups.duplicate_dno)]

In [72]:
lname = 'FRAM'
# df10.loc[~pd.isnull(df10.duplicate_dno), NAME_COLS+['dno', 'duplicate_dno']]
df10.loc[df10.clean_last_name==lname, NAME_COLS+['dno', 'duplicate_dno', 'clean_medical_school']]
uniques.loc[uniques.clean_last_name==lname, NAME_COLS+['dno', 'duplicate_dno', 'clean_medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,dno,duplicate_dno,clean_medical_school
24,DAVID,H,FRAM,1153.0,NaN,YALE UNIVERSITY SCHOOL OF MEDICINE


In [73]:
# it seems that dno does refer to unique person, so drop dups based on that 
# save this unique to pick
df10.to_pickle(os.path.join(PICKLE_DIR, 'unique_attendees.p'))

# to csv
df10.to_csv(os.path.join(ATT_DATA_DIR, 'unique_attendees.csv'))

In [ ]:
missing_eod_year = df10.loc[pd.isnull(df10.eod_year), :].sort_values('clean_last_name')

missing_eod_year[NAME_COLS+['clean_medical_school', 'dno']].to_csv(os.path.join(ATT_DATA_DIR, 'missing_eod_year.csv'), index=False)